# Complete Example: Imbalanced Data Best Practices

I'll create a comprehensive example that combines:
1. **Stratified CV** + **class_weight='balanced'**
2. **Target encoding inside Pipeline** (no leakage)
3. **ROC-AUC + PR-AUC** with prevalence baseline
4. **Calibration** for thresholding
5. **Optimal threshold selection**

---

## Complete Example

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (
    roc_auc_score, average_precision_score, 
    roc_curve, precision_recall_curve,
    classification_report, confusion_matrix,
    brier_score_loss, recall_score, precision_score, f1_score
)
from category_encoders import TargetEncoder

# ============================================================================
# STEP 1: CREATE IMBALANCED DATASET
# ============================================================================

print("="*70)
print("STEP 1: CREATE IMBALANCED DATASET")
print("="*70)

# Create imbalanced dataset (95% vs 5%)
X, y = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    weights=[0.95, 0.05],  # 95% class 0, 5% class 1
    flip_y=0.01,
    random_state=42
)

# Add high-cardinality categorical feature
# (Simulating customer_id, product_id, etc.)
np.random.seed(42)
n_categories = 100
categorical_feature = np.random.choice([f'CAT_{i}' for i in range(n_categories)], size=len(y))

# Make categories correlated with target
# Categories 0-30: Higher purchase rate
# Categories 31-99: Lower purchase rate
for i in range(len(y)):
    if y[i] == 1:  # Minority class
        # More likely to be in first 30 categories
        categorical_feature[i] = f'CAT_{np.random.randint(0, 40)}'
    else:  # Majority class
        # More likely to be in categories 20-99
        categorical_feature[i] = f'CAT_{np.random.randint(20, 100)}'

# Create DataFrame
X_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
X_df['customer_id'] = categorical_feature

print(f"Dataset shape: {X_df.shape}")
print(f"\nClass distribution:")
print(f"  Class 0 (majority): {(y == 0).sum()} ({(y == 0).mean():.1%})")
print(f"  Class 1 (minority): {(y == 1).sum()} ({(y == 1).mean():.1%})")

prevalence = (y == 1).mean()
imbalance_ratio = (y == 0).sum() / (y == 1).sum()
print(f"\nImbalance ratio: {imbalance_ratio:.1f}:1")
print(f"Prevalence (minority class rate): {prevalence:.2%}")

# ============================================================================
# STEP 2: TRAIN-TEST SPLIT (STRATIFIED!)
# ============================================================================

print("\n" + "="*70)
print("STEP 2: STRATIFIED TRAIN-TEST SPLIT")
print("="*70)

X_train, X_test, y_train, y_test = train_test_split(
    X_df, y,
    test_size=0.2,
    stratify=y,  # ✅ Maintain class distribution!
    random_state=42
)

print(f"\nTrain set:")
print(f"  Total: {len(y_train)}")
print(f"  Class 0: {(y_train == 0).sum()} ({(y_train == 0).mean():.1%})")
print(f"  Class 1: {(y_train == 1).sum()} ({(y_train == 1).mean():.1%})")

print(f"\nTest set:")
print(f"  Total: {len(y_test)}")
print(f"  Class 0: {(y_test == 0).sum()} ({(y_test == 0).mean():.1%})")
print(f"  Class 1: {(y_test == 1).sum()} ({(y_test == 1).mean():.1%})")

# ============================================================================
# STEP 3: BUILD PIPELINE (Target Encoding INSIDE to prevent leakage!)
# ============================================================================

print("\n" + "="*70)
print("STEP 3: BUILD PIPELINE (Target Encoding Inside)")
print("="*70)

# Separate numerical and categorical columns
numerical_cols = [col for col in X_train.columns if col.startswith('feature_')]
categorical_cols = ['customer_id']

print(f"\nNumerical features: {len(numerical_cols)}")
print(f"Categorical features: {len(categorical_cols)}")
print(f"Categorical cardinality: {X_train['customer_id'].nunique()} unique values")

# ✅ CORRECT: Target encoding in Pipeline
# This prevents data leakage during cross-validation!

from sklearn.compose import ColumnTransformer

# Build preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', TargetEncoder(), categorical_cols)  # ✅ Inside pipeline!
    ],
    remainder='drop'
)

# Complete pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        class_weight='balanced',  # ✅ Handle imbalance
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
])

print("\n✅ Pipeline created:")
print(pipeline)

# ============================================================================
# STEP 4: CROSS-VALIDATION (Stratified CV!)
# ============================================================================

print("\n" + "="*70)
print("STEP 4: CROSS-VALIDATION (Stratified)")
print("="*70)

# ✅ Stratified CV for imbalanced data
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Verify stratification works
print("\nVerifying stratification in each fold:")
for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
    y_train_fold = y_train.iloc[train_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    print(f"\nFold {fold_idx + 1}:")
    print(f"  Train: {len(train_idx)} samples, "
          f"Class 1: {(y_train_fold == 1).sum()} ({(y_train_fold == 1).mean():.1%})")
    print(f"  Val:   {len(val_idx)} samples, "
          f"Class 1: {(y_val_fold == 1).sum()} ({(y_val_fold == 1).mean():.1%})")

# Cross-validate with multiple metrics
print("\nRunning cross-validation...")

scoring = {
    'roc_auc': 'roc_auc',
    'pr_auc': 'average_precision',  # PR-AUC
    'recall': 'recall',
    'precision': 'precision',
    'f1': 'f1'
}

cv_results = cross_validate(
    pipeline, X_train, y_train,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=True
)

print("\n" + "-"*70)
print("CROSS-VALIDATION RESULTS")
print("-"*70)

for metric_name, metric_key in [('ROC-AUC', 'test_roc_auc'), 
                                  ('PR-AUC', 'test_pr_auc'),
                                  ('Recall', 'test_recall'),
                                  ('Precision', 'test_precision'),
                                  ('F1', 'test_f1')]:
    scores = cv_results[metric_key]
    print(f"{metric_name:12s}: {scores.mean():.4f} ± {scores.std():.4f}")

# Compare PR-AUC to prevalence baseline
pr_auc_mean = cv_results['test_pr_auc'].mean()
print(f"\nPR-AUC vs Baseline:")
print(f"  Model PR-AUC:      {pr_auc_mean:.4f}")
print(f"  Baseline (random): {prevalence:.4f}")
print(f"  Improvement:       {pr_auc_mean/prevalence:.1f}x better than random ✅")

# ============================================================================
# STEP 5: TRAIN FINAL MODEL & CALIBRATE
# ============================================================================

print("\n" + "="*70)
print("STEP 5: TRAIN FINAL MODEL & CALIBRATE")
print("="*70)

# Train uncalibrated model
print("\nTraining uncalibrated model...")
pipeline.fit(X_train, y_train)

# Apply calibration
print("Calibrating model...")
calibrated_model = CalibratedClassifierCV(
    pipeline,
    method='sigmoid',  # Platt scaling
    cv=cv,  # Use same CV strategy
    n_jobs=-1
)

calibrated_model.fit(X_train, y_train)
print("✅ Calibration complete")

# ============================================================================
# STEP 6: EVALUATE CALIBRATION QUALITY
# ============================================================================

print("\n" + "="*70)
print("STEP 6: EVALUATE CALIBRATION QUALITY")
print("="*70)

# Get predictions
y_proba_uncal = pipeline.predict_proba(X_test)[:, 1]
y_proba_cal = calibrated_model.predict_proba(X_test)[:, 1]

# Brier score (lower is better)
brier_uncal = brier_score_loss(y_test, y_proba_uncal)
brier_cal = brier_score_loss(y_test, y_proba_cal)

print(f"\nBrier Score (lower is better):")
print(f"  Uncalibrated: {brier_uncal:.4f}")
print(f"  Calibrated:   {brier_cal:.4f}")
print(f"  Improvement:  {brier_uncal - brier_cal:.4f}")

# Plot calibration curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calibration curve
fraction_pos_uncal, mean_pred_uncal = calibration_curve(
    y_test, y_proba_uncal, n_bins=10, strategy='uniform'
)
fraction_pos_cal, mean_pred_cal = calibration_curve(
    y_test, y_proba_cal, n_bins=10, strategy='uniform'
)

axes[0].plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
axes[0].plot(mean_pred_uncal, fraction_pos_uncal, 's-', 
             label=f'Before calibration (Brier={brier_uncal:.3f})', linewidth=2)
axes[0].plot(mean_pred_cal, fraction_pos_cal, 'o-', 
             label=f'After calibration (Brier={brier_cal:.3f})', linewidth=2)
axes[0].set_xlabel('Mean Predicted Probability', fontsize=12)
axes[0].set_ylabel('Fraction of Positives', fontsize=12)
axes[0].set_title('Calibration Curve', fontsize=14)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Probability distributions
axes[1].hist(y_proba_uncal, bins=20, alpha=0.5, label='Uncalibrated', edgecolor='black')
axes[1].hist(y_proba_cal, bins=20, alpha=0.5, label='Calibrated', edgecolor='black')
axes[1].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Default threshold')
axes[1].set_xlabel('Predicted Probability', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Probability Distributions', fontsize=14)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('calibration_analysis.png', dpi=150, bbox_inches='tight')
print("\n✅ Calibration plots saved to 'calibration_analysis.png'")
plt.show()

# ============================================================================
# STEP 7: EVALUATE WITH ROC-AUC AND PR-AUC
# ============================================================================

print("\n" + "="*70)
print("STEP 7: MODEL EVALUATION (Test Set)")
print("="*70)

# Calculate metrics
test_roc_auc = roc_auc_score(y_test, y_proba_cal)
test_pr_auc = average_precision_score(y_test, y_proba_cal)
test_prevalence = (y_test == 1).mean()

print(f"\nTest Set Metrics:")
print(f"  ROC-AUC:  {test_roc_auc:.4f}")
print(f"  PR-AUC:   {test_pr_auc:.4f}")
print(f"\nBaseline Comparison:")
print(f"  Prevalence (random baseline): {test_prevalence:.4f}")
print(f"  PR-AUC improvement: {test_pr_auc/test_prevalence:.1f}x over random ✅")

# Plot ROC and PR curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba_cal)
axes[0].plot(fpr, tpr, linewidth=2, label=f'Model (AUC = {test_roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random (AUC = 0.500)')
axes[0].set_xlabel('False Positive Rate', fontsize=12)
axes[0].set_ylabel('True Positive Rate', fontsize=12)
axes[0].set_title('ROC Curve', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# PR Curve
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba_cal)
axes[1].plot(recall_vals, precision_vals, linewidth=2, 
             label=f'Model (AP = {test_pr_auc:.3f})')
axes[1].axhline(y=test_prevalence, color='k', linestyle='--', linewidth=2,
                label=f'Random (AP = {test_prevalence:.3f})')
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('roc_pr_curves.png', dpi=150, bbox_inches='tight')
print("✅ ROC and PR curves saved to 'roc_pr_curves.png'")
plt.show()

# ============================================================================
# STEP 8: FIND OPTIMAL THRESHOLD
# ============================================================================

print("\n" + "="*70)
print("STEP 8: FIND OPTIMAL THRESHOLD")
print("="*70)

# Method 1: Maximize F1 Score
precision_vals, recall_vals, thresholds_pr = precision_recall_curve(y_test, y_proba_cal)

# Calculate F1 for each threshold
f1_scores = 2 * (precision_vals[:-1] * recall_vals[:-1]) / (precision_vals[:-1] + recall_vals[:-1] + 1e-10)
optimal_idx_f1 = np.argmax(f1_scores)
optimal_threshold_f1 = thresholds_pr[optimal_idx_f1]

print(f"\nMethod 1: Maximize F1 Score")
print(f"  Optimal threshold: {optimal_threshold_f1:.3f}")
print(f"  F1 score:          {f1_scores[optimal_idx_f1]:.4f}")
print(f"  Precision:         {precision_vals[optimal_idx_f1]:.4f}")
print(f"  Recall:            {recall_vals[optimal_idx_f1]:.4f}")

# Method 2: Youden's J statistic (maximize TPR - FPR)
fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba_cal)
j_scores = tpr - fpr
optimal_idx_j = np.argmax(j_scores)
optimal_threshold_j = thresholds_roc[optimal_idx_j]

print(f"\nMethod 2: Youden's J Statistic (ROC-based)")
print(f"  Optimal threshold: {optimal_threshold_j:.3f}")
print(f"  J statistic:       {j_scores[optimal_idx_j]:.4f}")
print(f"  TPR (Recall):      {tpr[optimal_idx_j]:.4f}")
print(f"  FPR:               {fpr[optimal_idx_j]:.4f}")

# Method 3: Target specific recall (e.g., 80%)
target_recall = 0.80
recall_diff = np.abs(recall_vals[:-1] - target_recall)
optimal_idx_recall = np.argmin(recall_diff)
optimal_threshold_recall = thresholds_pr[optimal_idx_recall]

print(f"\nMethod 3: Target Recall = {target_recall:.0%}")
print(f"  Optimal threshold: {optimal_threshold_recall:.3f}")
print(f"  Achieved recall:   {recall_vals[optimal_idx_recall]:.4f}")
print(f"  Precision:         {precision_vals[optimal_idx_recall]:.4f}")
print(f"  F1 score:          {f1_scores[optimal_idx_recall]:.4f}")

# Method 4: Cost-sensitive threshold
# Assume: Cost of False Negative = 10, Cost of False Positive = 1
cost_fp = 1
cost_fn = 10

def calculate_cost(threshold, y_true, y_proba, cost_fp, cost_fn):
    y_pred = (y_proba >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    total_cost = fp * cost_fp + fn * cost_fn
    return total_cost

# Try different thresholds
threshold_range = np.linspace(0.01, 0.99, 100)
costs = [calculate_cost(t, y_test, y_proba_cal, cost_fp, cost_fn) for t in threshold_range]
optimal_idx_cost = np.argmin(costs)
optimal_threshold_cost = threshold_range[optimal_idx_cost]

print(f"\nMethod 4: Cost-Sensitive (FP cost={cost_fp}, FN cost={cost_fn})")
print(f"  Optimal threshold: {optimal_threshold_cost:.3f}")
print(f"  Total cost:        {costs[optimal_idx_cost]:.0f}")

# Visualize threshold selection
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# F1 vs Threshold
axes[0, 0].plot(thresholds_pr, f1_scores, linewidth=2)
axes[0, 0].axvline(optimal_threshold_f1, color='red', linestyle='--', 
                   linewidth=2, label=f'Optimal = {optimal_threshold_f1:.3f}')
axes[0, 0].axvline(0.5, color='gray', linestyle=':', linewidth=2, label='Default = 0.5')
axes[0, 0].set_xlabel('Threshold', fontsize=11)
axes[0, 0].set_ylabel('F1 Score', fontsize=11)
axes[0, 0].set_title('Method 1: Maximize F1 Score', fontsize=12)
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Youden's J vs Threshold
axes[0, 1].plot(thresholds_roc, j_scores, linewidth=2)
axes[0, 1].axvline(optimal_threshold_j, color='red', linestyle='--', 
                   linewidth=2, label=f'Optimal = {optimal_threshold_j:.3f}')
axes[0, 1].axvline(0.5, color='gray', linestyle=':', linewidth=2, label='Default = 0.5')
axes[0, 1].set_xlabel('Threshold', fontsize=11)
axes[0, 1].set_ylabel("Youden's J (TPR - FPR)", fontsize=11)
axes[0, 1].set_title("Method 2: Youden's J Statistic", fontsize=12)
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Precision-Recall vs Threshold
axes[1, 0].plot(thresholds_pr, precision_vals[:-1], label='Precision', linewidth=2)
axes[1, 0].plot(thresholds_pr, recall_vals[:-1], label='Recall', linewidth=2)
axes[1, 0].axvline(optimal_threshold_recall, color='red', linestyle='--', 
                   linewidth=2, label=f'Target Recall = {target_recall:.0%}')
axes[1, 0].axvline(0.5, color='gray', linestyle=':', linewidth=2, label='Default = 0.5')
axes[1, 0].set_xlabel('Threshold', fontsize=11)
axes[1, 0].set_ylabel('Score', fontsize=11)
axes[1, 0].set_title(f'Method 3: Target Recall = {target_recall:.0%}', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Cost vs Threshold
axes[1, 1].plot(threshold_range, costs, linewidth=2)
axes[1, 1].axvline(optimal_threshold_cost, color='red', linestyle='--', 
                   linewidth=2, label=f'Optimal = {optimal_threshold_cost:.3f}')
axes[1, 1].axvline(0.5, color='gray', linestyle=':', linewidth=2, label='Default = 0.5')
axes[1, 1].set_xlabel('Threshold', fontsize=11)
axes[1, 1].set_ylabel('Total Cost', fontsize=11)
axes[1, 1].set_title(f'Method 4: Cost-Sensitive (FP={cost_fp}, FN={cost_fn})', fontsize=12)
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('threshold_selection.png', dpi=150, bbox_inches='tight')
print("\n✅ Threshold selection plots saved to 'threshold_selection.png'")
plt.show()

# ============================================================================
# STEP 9: COMPARE THRESHOLDS
# ============================================================================

print("\n" + "="*70)
print("STEP 9: COMPARE DIFFERENT THRESHOLDS")
print("="*70)

thresholds_to_compare = {
    'Default (0.5)': 0.5,
    'Maximize F1': optimal_threshold_f1,
    "Youden's J": optimal_threshold_j,
    f'Target Recall {target_recall:.0%}': optimal_threshold_recall,
    'Cost-Sensitive': optimal_threshold_cost
}

print("\nThreshold Comparison:")
print("-" * 90)
print(f"{'Method':<25} {'Threshold':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'Cost':>10}")
print("-" * 90)

for method_name, threshold in thresholds_to_compare.items():
    y_pred = (y_proba_cal >= threshold).astype(int)
    
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cost = calculate_cost(threshold, y_test, y_proba_cal, cost_fp, cost_fn)
    
    print(f"{method_name:<25} {threshold:>10.3f} {prec:>10.4f} {rec:>10.4f} {f1:>10.4f} {cost:>10.0f}")

print("-" * 90)

# ============================================================================
# STEP 10: FINAL EVALUATION WITH OPTIMAL THRESHOLD
# ============================================================================

print("\n" + "="*70)
print("STEP 10: FINAL EVALUATION (Using F1-Optimal Threshold)")
print("="*70)

# Use F1-optimal threshold for final evaluation
final_threshold = optimal_threshold_f1
y_pred_final = (y_proba_cal >= final_threshold).astype(int)

print(f"\nFinal Threshold: {final_threshold:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_final, 
                          target_names=['Majority (0)', 'Minority (1)']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred_final)
print(cm)

tn, fp, fn, tp = cm.ravel()
print(f"\nDetailed Breakdown:")
print(f"  True Negatives:  {tn:5d}")
print(f"  False Positives: {fp:5d}")
print(f"  False Negatives: {fn:5d}")
print(f"  True Positives:  {tp:5d}")

print(f"\nMinority Class Metrics:")
print(f"  Precision: {tp / (tp + fp):.4f} ({tp} / {tp + fp})")
print(f"  Recall:    {tp / (tp + fn):.4f} ({tp} / {tp + fn})")
print(f"  F1 Score:  {2*tp / (2*tp + fp + fn):.4f}")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*70)
print("SUMMARY: BEST PRACTICES FOR IMBALANCED DATA")
print("="*70)

summary = f"""
✅ Dataset:
   • Imbalance ratio: {imbalance_ratio:.1f}:1
   • Prevalence: {prevalence:.2%}

✅ Data Splitting:
   • Used stratified train-test split
   • Maintains class distribution in both sets

✅ Cross-Validation:
   • StratifiedKFold (5 folds)
   • Fair evaluation across all folds

✅ Pipeline (Prevents Data Leakage):
   • Target encoding INSIDE pipeline
   • Fitted separately on each CV fold
   • No leakage from validation data

✅ Class Imbalance Handling:
   • class_weight='balanced' in RandomForest
   • Adjusts loss function to focus on minority class

✅ Calibration:
   • Applied Platt scaling (sigmoid method)
   • Brier score improved: {brier_uncal:.4f} → {brier_cal:.4f}
   • Enables reliable threshold selection

✅ Evaluation Metrics:
   • ROC-AUC: {test_roc_auc:.4f} (discrimination ability)
   • PR-AUC:  {test_pr_auc:.4f} ({test_pr_auc/test_prevalence:.1f}x better than random)
   • Prevalence baseline: {test_prevalence:.4f}

✅ Threshold Selection:
   • Explored 4 methods
   • Chose F1-optimal: {final_threshold:.3f}
   • Default 0.5 threshold is often suboptimal!

✅ Final Performance (threshold={final_threshold:.3f}):
   • Precision: {tp / (tp + fp):.4f}
   • Recall:    {tp / (tp + fn):.4f}
   • F1 Score:  {2*tp / (2*tp + fp + fn):.4f}
"""

print(summary)

print("="*70)
print("WORKFLOW COMPLETE! 🎉")
print("="*70)
```

---

## Deep Dive: Threshold Selection Methods

Let me explain each threshold selection method in detail:

### Method 1: Maximize F1 Score

**When to use:** Balanced importance of precision and recall

```python
def find_optimal_threshold_f1(y_true, y_proba):
    """
    Find threshold that maximizes F1 score
    """
    precision_vals, recall_vals, thresholds = precision_recall_curve(y_true, y_proba)
    
    # F1 = 2 * (precision * recall) / (precision + recall)
    f1_scores = 2 * (precision_vals[:-1] * recall_vals[:-1]) / \
                (precision_vals[:-1] + recall_vals[:-1] + 1e-10)
    
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    return optimal_threshold, f1_scores[optimal_idx]

threshold, max_f1 = find_optimal_threshold_f1(y_test, y_proba_cal)
print(f"Optimal threshold: {threshold:.3f}, Max F1: {max_f1:.4f}")
```

**Pros:**
- ✅ Balances precision and recall
- ✅ Single number to optimize
- ✅ Works well for general use cases

**Cons:**
- ❌ Assumes equal importance of precision and recall
- ❌ May not suit business constraints

---

### Method 2: Youden's J Statistic

**When to use:** Want to maximize true positive rate while minimizing false positive rate

```python
def find_optimal_threshold_youden(y_true, y_proba):
    """
    Find threshold that maximizes Youden's J = TPR - FPR
    Also called "sensitivity + specificity - 1"
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    
    # J = TPR - FPR = Sensitivity + Specificity - 1
    j_scores = tpr - fpr
    
    optimal_idx = np.argmax(j_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    return optimal_threshold, j_scores[optimal_idx], tpr[optimal_idx], fpr[optimal_idx]

threshold, j_score, tpr, fpr = find_optimal_threshold_youden(y_test, y_proba_cal)
print(f"Optimal threshold: {threshold:.3f}")
print(f"J statistic: {j_score:.4f}")
print(f"TPR: {tpr:.4f}, FPR: {fpr:.4f}")
```

**Interpretation:**
- **J = 0**: No better than random
- **J = 1**: Perfect classifier
- **J = 0.5**: Good practical performance

**Pros:**
- ✅ Based on ROC curve (well-understood)
- ✅ Maximizes overall classification performance
- ✅ Accounts for both sensitivity and specificity

**Cons:**
- ❌ Assumes equal importance of TPR and FPR
- ❌ May not reflect business costs

---

### Method 3: Target Specific Recall

**When to use:** Business requires minimum recall (e.g., must catch 80% of fraud)

```python
def find_threshold_for_target_recall(y_true, y_proba, target_recall=0.80):
    """
    Find threshold that achieves target recall
    while maximizing precision
    """
    precision_vals, recall_vals, thresholds = precision_recall_curve(y_true, y_proba)
    
    # Find thresholds where recall >= target
    valid_indices = np.where(recall_vals[:-1] >= target_recall)[0]
    
    if len(valid_indices) == 0:
        print(f"Warning: Cannot achieve {target_recall:.0%} recall")
        return None, None, None
    
    # Among valid thresholds, pick one with highest precision
    optimal_idx = valid_indices[np.argmax(precision_vals[:-1][valid_indices])]
    optimal_threshold = thresholds[optimal_idx]
    
    return (optimal_threshold, 
            precision_vals[optimal_idx], 
            recall_vals[optimal_idx])

threshold, prec, rec = find_threshold_for_target_recall(
    y_test, y_proba_cal, target_recall=0.80
)

print(f"Threshold for 80% recall: {threshold:.3f}")
print(f"Achieved - Precision: {prec:.4f}, Recall: {rec:.4f}")
```

**Use cases:**
- **Fraud detection**: Must catch ≥90% of fraud (high recall)
- **Disease screening**: Must catch ≥95% of cases (high recall)
- **Spam filtering**: Must not miss ≥80% of spam (high recall)

**Similarly, you can target precision:**

```python
def find_threshold_for_target_precision(y_true, y_proba, target_precision=0.90):
    """
    Find threshold that achieves target precision
    while maximizing recall
    """
    precision_vals, recall_vals, thresholds = precision_recall_curve(y_true, y_proba)
    
    # Find thresholds where precision >= target
    valid_indices = np.where(precision_vals[:-1] >= target_precision)[0]
    
    if len(valid_indices) == 0:
        print(f"Warning: Cannot achieve {target_precision:.0%} precision")
        return None, None, None
    
    # Among valid thresholds, pick one with highest recall
    optimal_idx = valid_indices[np.argmax(recall_vals[:-1][valid_indices])]
    optimal_threshold = thresholds[optimal_idx]
    
    return (optimal_threshold, 
            precision_vals[optimal_idx], 
            recall_vals[optimal_idx])
```

---

### Method 4: Cost-Sensitive Threshold

**When to use:** Different costs for false positives vs false negatives

```python
def find_optimal_threshold_cost(y_true, y_proba, cost_fp=1, cost_fn=10):
    """
    Find threshold that minimizes total cost
    
    Cost = (FP × cost_fp) + (FN × cost_fn)
    """
    thresholds = np.linspace(0.01, 0.99, 200)
    costs = []
    
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        
        total_cost = fp * cost_fp + fn * cost_fn
        costs.append(total_cost)
    
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    min_cost = costs[optimal_idx]
    
    return optimal_threshold, min_cost, costs, thresholds

# Example: Missing fraud (FN) costs 10x more than false alarm (FP)
threshold, min_cost, costs, thresholds = find_optimal_threshold_cost(
    y_test, y_proba_cal, cost_fp=1, cost_fn=10
)

print(f"Optimal threshold: {threshold:.3f}")
print(f"Minimum cost: {min_cost:.0f}")

# Plot cost vs threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, costs, linewidth=2)
plt.axvline(threshold, color='red', linestyle='--', 
            label=f'Optimal = {threshold:.3f}')
plt.xlabel('Threshold')
plt.ylabel('Total Cost')
plt.title(f'Cost-Sensitive Threshold (FP cost={1}, FN cost={10})')
plt.legend()
plt.grid(alpha=0.3)
plt.show()
```

**Real-world cost examples:**

```python
# Fraud detection
# FN (miss fraud): Lose $500 average transaction
# FP (false alarm): $10 investigation cost
cost_fp = 10
cost_fn = 500

# Medical diagnosis
# FN (miss disease): Patient dies, $1,000,000 lawsuit
# FP (false positive): $1,000 additional tests
cost_fp = 1000
cost_fn = 1000000

# Email spam
# FN (miss spam): 1 minute wasted
# FP (good email marked spam): 10 minutes to recover
cost_fp = 10
cost_fn = 1
```

---

### Method 5: F-Beta Score

**When to use:** Want to weight recall β times more than precision

```python
from sklearn.metrics import fbeta_score

def find_optimal_threshold_fbeta(y_true, y_proba, beta=2.0):
    """
    Find threshold that maximizes F-beta score
    
    beta > 1: Favors recall
    beta < 1: Favors precision
    beta = 1: F1 score (balanced)
    """
    precision_vals, recall_vals, thresholds = precision_recall_curve(y_true, y_proba)
    
    # F-beta = (1 + beta²) * (precision * recall) / (beta² * precision + recall)
    fbeta_scores = (1 + beta**2) * (precision_vals[:-1] * recall_vals[:-1]) / \
                   (beta**2 * precision_vals[:-1] + recall_vals[:-1] + 1e-10)
    
    optimal_idx = np.argmax(fbeta_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    return optimal_threshold, fbeta_scores[optimal_idx]

# F2 score: Recall is 2x more important than precision
threshold_f2, max_f2 = find_optimal_threshold_fbeta(y_test, y_proba_cal, beta=2.0)
print(f"F2-optimal threshold: {threshold_f2:.3f}, Max F2: {max_f2:.4f}")

# F0.5 score: Precision is 2x more important than recall
threshold_f05, max_f05 = find_optimal_threshold_fbeta(y_test, y_proba_cal, beta=0.5)
print(f"F0.5-optimal threshold: {threshold_f05:.3f}, Max F0.5: {max_f05:.4f}")
```

---

## Decision Framework for Threshold Selection

```
START: Which threshold method?
│
├─ Equal importance of precision & recall?
│   YES → Use F1 Score (Method 1) ✅
│
├─ Business requires minimum recall (e.g., 80%)?
│   YES → Use Target Recall (Method 3) ✅
│
├─ Business requires minimum precision (e.g., 90%)?
│   YES → Use Target Precision (Method 3 variant) ✅
│
├─ Know exact costs of FP and FN?
│   YES → Use Cost-Sensitive (Method 4) ✅
│
├─ Recall more important than precision?
│   YES → Use F-beta with beta > 1 (Method 5) ✅
│
├─ Precision more important than recall?
│   YES → Use F-beta with beta < 1 (Method 5) ✅
│
└─ No specific requirements?
    → Use Youden's J (Method 2) ✅
```

---

## Practical Tips

1. **Always calibrate before threshold tuning**
   - Uncalibrated probabilities → Wrong thresholds
   - Calibrate first, then find threshold

2. **Validate on separate dataset**
   - Find threshold on validation set
   - Test final threshold on test set
   - Prevents overfitting to threshold

3. **Plot threshold vs metrics**
   - Visualize trade-offs
   - Understand sensitivity to threshold changes

4. **Document your choice**
   - Why this threshold?
   - What trade-offs were made?
   - Business justification

5. **Monitor in production**
   - Threshold may need adjustment
   - Data distribution changes over time

Want me to show:
1. **How to implement threshold selection in production** (save/load thresholds)?
2. **Advanced: Multi-class threshold optimization**?
3. **How to handle threshold drift over time**?